# Create H, A, T

In [74]:
# import pandas as pd
# df = pd.read_csv('./datasets/raw/tdec.csv')

In [75]:
# dfs = df.sample(200)

In [76]:
# import stanza

# nlp = stanza.Pipeline(
#     lang="id",
#     processors="tokenize,pos,lemma",
#     download_method=stanza.DownloadMethod.REUSE_RESOURCES,
# )

In [77]:
# H = dfs.filter(['headline'], axis=1)
# H.columns = ['H']

# A = dfs.filter(['body'], axis=1)
# A.columns = ['A']
# A['A'] = dfs['headline'] + '. ' + A['A']

# T = dfs.filter(['body'], axis=1)
# T.columns = ['T']
# T['T'] = dfs['headline'] + '. ' + T['T']
# in_T = list(T['T'].apply(lambda h: stanza.Document([], text=h)))
# out_T = nlp(in_T)
# T = pd.DataFrame([s.text for a in out_T for s in a.sentences], columns=['T'])

In [78]:
# H.to_csv('./datasets/small/H.csv', index=False)
# A.to_csv('./datasets/small/A.csv', index=False)
# T.to_csv('./datasets/small/S.csv', index=False)

# Create Variants

In [12]:
import pandas as pd

H = pd.read_csv('./datasets/small/H.csv')
A = pd.read_csv('./datasets/small/A.csv')
T = pd.read_csv('./datasets/small/S.csv')

In [13]:
import stanza

nlp = stanza.Pipeline(
    lang="id",
    processors="tokenize,pos,lemma",
    download_method=stanza.DownloadMethod.REUSE_RESOURCES,
)

2023-08-15 17:36:44 WARNING: Language id package default expects mwt, which has been added


2023-08-15 17:36:45 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |

2023-08-15 17:36:45 INFO: Using device: cpu
2023-08-15 17:36:45 INFO: Loading: tokenize
2023-08-15 17:36:45 INFO: Loading: mwt
2023-08-15 17:36:45 INFO: Loading: pos
2023-08-15 17:36:45 INFO: Loading: lemma
2023-08-15 17:36:45 INFO: Done loading processors!


In [14]:
allowed_pos = ['NOUN', 'PROPN', 'VERB', 'X']

def preprocess_clw(out_df, c=False, l=False, w=False):
    r = [] # result
    for d in out_df:
        tokens = []
        for s in d.sentences:
            for token in s.words:
                _token = token.text
                if w:
                    if token.upos not in allowed_pos:
                        continue
                if l:
                    _token = token.lemma if token.lemma else _token
                if c:
                    _token = _token if l else token.text.lower()
                tokens.append(_token)
        r.append(' '.join(tokens))
    return r

In [15]:
allowed_pos = ['NOUN', 'PROPN', 'VERB', 'X']

def preprocess(df_column):
    r = {} # result
    in_df = list(df_column.apply(lambda x: stanza.Document([], text=x)))
    out_df = nlp(in_df)
    r['T'] = preprocess_clw(out_df)
    r['C'] = preprocess_clw(out_df, c=True)
    r['CL'] = preprocess_clw(out_df, c=True, l=True)
    r['CLW'] = preprocess_clw(out_df, c=True, l=True, w=True)
    r['CW'] = preprocess_clw(out_df, c=True, w=True)
    r['L'] = preprocess_clw(out_df, l=True)
    r['LW'] = preprocess_clw(out_df, l=True, w=True)
    r['W'] = preprocess_clw(out_df, w=True)
    return r

In [16]:
HCLW = pd.DataFrame(preprocess(H['H']))
ACLW = pd.DataFrame(preprocess(A['A']))
TCLW = pd.DataFrame(preprocess(T['T']))

In [29]:
# HCLW.to_csv('./datasets/small/HCLW.csv', index=False)
# ACLW.to_csv('./datasets/small/ACLW.csv', index=False)
# TCLW.to_csv('./datasets/small/SCLW.csv', index=False)